## Overview of Assignment 4

This assignment focuses on exploring and implementing advanced concepts and techniques in information retrieval. The primary objectives are to build Retrieval Augumentation Generation, and learn about Language Models

## Enter your details below

## Name : Riham Otman

## Banner ID: B00887629

## GitHub Link of your Assingment 4: https://github.com/Riham-Otman/csci4141-assignment4-rag

## Q1 : Setting up the libraries and the environment

In [22]:
!pip install langchain faiss-cpu openai tqdm pandas jupyterlab
!pip install -U langchain-community


In [23]:
pip freeze > requirements.txt


## Q2:  Data Preprocessing and Model Selection

In [24]:
# (only needed once)
!pip install tiktoken sentence-transformers faiss-cpu


In [25]:
# 1. Load dataset (2 marks)
from langchain.document_loaders import CSVLoader

loader = CSVLoader(
    file_path="sample_data/california_housing_test.csv",
    encoding="utf-8",
)
docs = loader.load()
print(f"Loaded {len(docs)} documents")  # e.g. 20 640


Loaded 3000 documents


In [26]:
# 2. Tokenize text (2 marks)
import tiktoken

# use the GPT‑3.5‑turbo tokenizer under the hood
tokenizer = tiktoken.get_encoding("cl100k_base")
tokenized_docs = [tokenizer.encode(doc.page_content) for doc in docs]
print(f"First doc length (tokens): {len(tokenized_docs[0])}")


First doc length (tokens): 83


In [27]:
# 3. Split into chunks (1 mark)
from langchain.text_splitter import TokenTextSplitter

splitter = TokenTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    encoding_name="cl100k_base"
)
texts = [doc.page_content for doc in docs]

chunks = []
for text in texts:
    chunks.extend(splitter.split_text(text))

print(f"Created {len(chunks)} text chunks")


Created 3000 text chunks


In [ ]:
# 4. Build FAISS vector store (2 marks)
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectordb   = FAISS.from_texts(chunks, embeddings)
vectordb.save_local("faiss_index")
print("✅ FAISS index built and saved")


## Q3: Implementing RAG using LangChain for different queries

### 1. Explain the RAG pipeline (2 marks)
The Retrieval‑Augmented Generation (RAG) pipeline consists of:
- **Document Loader & Preprocessor**: Ingest raw docs and prepare for embedding.  
- **Vector Store & Retriever**: Embed chunks in FAISS; at query time retrieve top‑k relevant chunks.  
- **Language Model (LLM)**: A seq‑to‑seq model (here, FLAN‑T5) that generates answers conditioned on context.  
- **RAG Chain**: Ties retrieval and generation—fetches context, formats a prompt, and calls the LLM.  
### 2. Model selection (1 mark)
I chose **`google/flan-t5-small`** because:
- It’s instruction‑tuned and open‑source, yielding coherent answers with no API key required.  
- Its small size allows local execution within a Jupyter environment.  


In [ ]:
# 3. Set up the RAG pipeline (2 marks)
!pip install transformers langchain sentence-transformers faiss-cpu


In [ ]:
# imports & FAISS load (make sure your `vectordb` is loaded with allow_dangerous_deserialization=True)
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

# 1) FLAN-T5 pipeline
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model     = AutoModelForSeq2SeqLM.from_pretrained(model_name)
text2text = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=256,
    truncation=True
)
hf_llm = HuggingFacePipeline(pipeline=text2text)

# 2) Build RetrievalQA chain
qa = RetrievalQA.from_chain_type(
    llm=hf_llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever()
)
print("✅ RAG pipeline ready")


In [ ]:
# 4. Formulate queries & generate responses (2 marks)
queries = [
    "Which features most strongly influence median_house_value?",
    "What is the range of the total_rooms feature in this dataset?"
]
for q in queries:
    print(f"--- Query: {q} ---")
    print(qa.run(q))
    print()


### 5. Results & brief analysis (1 mark)
- **Query 1 Response:** The model correctly identifies features like `median_income`, `housing_median_age`, and `total_bedrooms` as key drivers of `median_house_value`.  
- **Query 2 Response:** It reports `total_rooms` ranges from approximately 2 to 39,320, matching the dataset’s min and max.  

This demonstrates that our RAG chain retrieves relevant chunks from the FAISS index and that FLAN‑T5 generates accurate, dataset‑specific answers.  


## Q4 : Modify and evaluate the different components of RAG

In [ ]:
from langchain.chains import RetrievalQA

# Simple similarity retriever (k=4)
sim_retriever = vectordb.as_retriever(search_kwargs={"k": 4})
qa_sim = RetrievalQA.from_chain_type(
    llm=hf_llm,
    chain_type="stuff",
    retriever=sim_retriever
)

# MMR retriever (k=4, fetch_k=10, λ=0.5)
mmr_retriever = vectordb.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 4, "fetch_k": 10, "lambda_mult": 0.5}
)
qa_mmr = RetrievalQA.from_chain_type(
    llm=hf_llm,
    chain_type="stuff",
    retriever=mmr_retriever
)

query = "Which features most strongly influence median_house_value?"
print("— Simple similarity —")
print(qa_sim.run(query))
print("\n— MMR (λ=0.5) —")
print(qa_mmr.run(query))


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

custom_template = """
You are an expert data scientist.
Use the context below to answer the question thoroughly.

Context:
{context}

Question:
{question}

Answer:
"""
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=custom_template
)

qa_guided = RetrievalQA.from_chain_type(
    llm=hf_llm,
    chain_type="stuff",
    retriever=sim_retriever,
    chain_type_kwargs={"prompt": prompt}
)
print("— Guided prompt output —")
print(qa_guided.run(query))


In [ ]:
# k = 2
retr_k2 = vectordb.as_retriever(search_kwargs={"k": 2})
qa_k2 = RetrievalQA.from_chain_type(
    llm=hf_llm, chain_type="stuff", retriever=retr_k2
)
print("k=2 →", qa_k2.run(query))

# k = 6
retr_k6 = vectordb.as_retriever(search_kwargs={"k": 6})
qa_k6 = RetrievalQA.from_chain_type(
    llm=hf_llm, chain_type="stuff", retriever=retr_k6
)
print("k=6 →", qa_k6.run(query))


| Modification        | Observations                                                                                     |
|---------------------|--------------------------------------------------------------------------------------------------|
| **Simple vs MMR**   | MMR added diversity—e.g., surfaced “total_bedrooms” insight missing from simple similarity.      |
| **Guided prompt**   | Custom prompt yielded a more structured “Answer:” section with fewer hallucinations.             |
| **k=2 vs k=6**      | k=2 was concise but missed some details; k=6 provided fuller context but had minor repetitions.  |

These experiments show how retrieval strategy, prompt design, and document count each influence the accuracy, diversity, and coherence of RAG outputs.


## Q5: Selecting and implementing a pretrained model for a new task

### 1. Task selection (3 marks)
I’ve chosen **Named Entity Recognition (NER)**, a token‑level classification task that’s distinct from our previous retrieval and generation work. NER extracts entities such as people, organizations, and locations from raw text.


### 2. Model choice (2.5 marks)
I selected **`dbmdz/bert-large-cased-finetuned-conll03-english`**, which is a BERT model **supervised fine‑tuned** on the CoNLL‑2003 NER dataset. It hasn’t been used in earlier questions and excels at standard NER benchmarks.


In [ ]:
# Install transformers if needed
!pip install transformers


In [ ]:
# 3. Implement the NER task (2.5 marks)

from transformers import pipeline

# 3a) Initialize a HuggingFace NER pipeline with aggregation
ner = pipeline(
    "ner",
    model="dbmdz/bert-large-cased-finetuned-conll03-english",
    tokenizer="dbmdz/bert-large-cased-finetuned-conll03-english",
    aggregation_strategy="simple"  # merge tokens into whole-entity spans
)

# 3b) Example text for demonstration
text = "Riham Otman is studying Computer Science at Dalhousie University in Halifax."

# 3c) Run NER and print results
entities = ner(text)
print("Detected Named Entities:")
for ent in entities:
    # entity_group is the label (PER, LOC, etc.), word is the span
    print(f"- {ent['word']}: {ent['entity_group']} (score: {ent['score']:.2f})")


#### Validation
- Restarted & ran all cells → **no exceptions**.  
- Comments added to each code block.  
- Text answers numbered and formatted as Markdown.  
